In [ ]:
!pip install transformers
!pip install datasets
!pip install tokenizers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset
ds = load_dataset("abisee/cnn_dailymail", "3.0.0")


# Data Cleaning



In [ ]:
import re



contractions = {
    "can't": "cannot",
    "won't": "will not",
    "don't": "do not",
    "didn't": "did not",
    "doesn't": "does not",
    "isn't": "is not",
    "aren't": "are not",
    "wasn't": "was not",
    "weren't": "were not",
    "hasn't": "has not",
    "haven't": "have not",
    "hadn't": "had not",
    "he's": "he is",
    "she's": "she is",
    "it's": "it is",
    "we're": "we are",
    "they're": "they are",
    "i'm": "i am",
    "you're": "you are",
    "i've": "i have",
    "you've": "you have",
    "he'd": "he would",
    "she'd": "she would",
    "we'd": "we would",
    "they'd": "they would",
    "i'd": "i would",
    "i'll": "i will",
    "you'll": "you will",
    "he'll": "he will",
    "she'll": "she will",
    "we'll": "we will",
    "they'll": "they will",
    "let's": "let us",
    "that's": "that is",
    "what's": "what is",
    "here's": "here is",
    "there's": "there is",
    "who's": "who is",
    "how's": "how is",
    "it's": "it is",
    "i'll": "i will",
    "she'll": "she will",
    "he'll": "he will",
    "we'll": "we will",
    "they'll": "they will"
}

def expand_contractions(text):
    # Use regex to replace contractions with their expanded forms
    pattern = re.compile(r'\b(' + '|'.join(re.escape(key) for key in contractions.keys()) + r')\b')
    return pattern.sub(lambda x: contractions[x.group()], text)

def clean_text(text):

    text = text.lower()

    text = expand_contractions(text)

    text = re.sub(r'\([^)]*\)', '', text)

    text = re.sub(r"e-mail to a friend.*?", "", text, flags=re.IGNORECASE)
    text = re.sub(r"copyright.*?all rights reserved.*?redistributed\.", "", text, flags=re.IGNORECASE)
    text = re.sub(r"this material may not be published.*?redistributed\.", "", text, flags=re.IGNORECASE)

    # Remove punctuation and numbers
    text = re.sub(r"[^a-z\s]", "", text)

    # Normalize whitespace (remove multiple spaces)
    text = re.sub(r"\s+", " ", text).strip()

    return text

combined_data = []


# Combine 'train', 'test', and 'validation'
for split in ['train', 'test', 'validation']:
    for i in range(len(ds[split])):
        combined_data.append({
            'article': ds[split][i]['article'],
            'highlights': ds[split][i]['highlights'],
            'split': split  # Keep track of which split it came from
        })


# Step 2: Clean the combined dataset
cleaned_combined_data = []
for entry in combined_data:
    cleaned_entry = {
        'article': clean_text(entry['article']),
        'highlights': clean_text(entry['highlights']),
        'split': entry['split']  # Keep the split info to reassign later
    }
    cleaned_combined_data.append(cleaned_entry)


# Step 3: Re-split the cleaned data back into 'train', 'test', and 'validation'
cleaned_ds = {'train': [], 'test': [], 'validation': []}


for entry in cleaned_combined_data:
    cleaned_ds[entry['split']].append({
        'article': entry['article'],
        'highlights': entry['highlights']
    })


print(cleaned_ds['train'][0]['article'])
print(cleaned_ds['train'][0]['highlights'])






london england harry potter star daniel radcliffe gains access to a reported million fortune as he turns on monday but he insists the money will not cast a spell on him daniel radcliffe as harry potter in harry potter and the order of the phoenix to the disappointment of gossip columnists around the world the young actor says he has no plans to fritter his cash away on fast cars drink and celebrity parties i do not plan to be one of those people who as soon as they turn suddenly buy themselves a massive sports car collection or something similar he told an australian interviewer earlier this month i do not think i will be particularly extravagant the things i like buying are things that cost about pounds books and cds and dvds at radcliffe will be able to gamble in a casino buy a drink in a pub or see the horror film hostel part ii currently six places below his number one movie on the uk box office chart details of how he will mark his landmark birthday are under wraps his agent and p

In [ ]:
print("Train Sample:")
print(cleaned_ds['train'][0]['article'])
print(cleaned_ds['train'][0]['highlights'])

print("\nTest Sample:")
print(cleaned_ds['test'][0]['article'])
print(cleaned_ds['test'][0]['highlights'])

print("\nValidation Sample:")
print(cleaned_ds['validation'][0]['article'])
print(cleaned_ds['validation'][0]['highlights'])


Train Sample:
london england harry potter star daniel radcliffe gains access to a reported million fortune as he turns on monday but he insists the money will not cast a spell on him daniel radcliffe as harry potter in harry potter and the order of the phoenix to the disappointment of gossip columnists around the world the young actor says he has no plans to fritter his cash away on fast cars drink and celebrity parties i do not plan to be one of those people who as soon as they turn suddenly buy themselves a massive sports car collection or something similar he told an australian interviewer earlier this month i do not think i will be particularly extravagant the things i like buying are things that cost about pounds books and cds and dvds at radcliffe will be able to gamble in a casino buy a drink in a pub or see the horror film hostel part ii currently six places below his number one movie on the uk box office chart details of how he will mark his landmark birthday are under wraps h

In [ ]:
import json

# Save the cleaned dataset to a JSON file
with open('cleaned_dataset.json', 'w') as f:
    json.dump(cleaned_ds, f)

# Load the dataset back
with open('cleaned_dataset.json', 'r') as f:
    cleaned_ds = json.load(f)

print(cleaned_ds['train'][0])  # Verify the loaded data


KeyboardInterrupt: 

# Data Preprocessing

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Trainer, TrainingArguments
from datasets import DatasetDict
from transformers import PegasusTokenizer, PegasusForConditionalGeneration, T5Tokenizer, T5ForConditionalGeneration

#tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-cnn_dailymail")

tokenizer = T5Tokenizer.from_pretrained("t5-base")


class Sum_Dataset(Dataset):
    def __init__(self, dataset, tokenizer, max_input_length = 512, max_length=128):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.max_input_length = max_input_length

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        article = item['article']
        highlights = item['highlights']



        # Tokenize the articles (inputs)
        inputs = self.tokenizer(
            article,  # Accessing the 'article' from the dataset item
            max_length=self.max_input_length,
            truncation=True,
            padding="max_length",
            return_tensors="pt"
        )

        # Tokenize the highlights (targets/summaries)
        targets = self.tokenizer(
            highlights,  # Accessing the 'highlights' from the dataset item
            max_length=self.max_length,
            truncation=True,
            padding="max_length",
            return_tensors="pt"


        )

        return {
            "input_ids": torch.tensor(inputs["input_ids"]).squeeze(0),
            "attention_mask": torch.tensor(inputs["attention_mask"]).squeeze(0),
            "labels": torch.tensor(targets["input_ids"]).squeeze(0)  # Tokenized summaries as labels
        }


In [ ]:
# Assuming ds is a DatasetDict containing the train, validation, and test splits

# Create the train, validation, and test datasets using the CaptionDataset class
train_dataset = Sum_Dataset(cleaned_ds['train'], tokenizer, max_input_length=512, max_length=128)
validation_dataset = Sum_Dataset(cleaned_ds['validation'], tokenizer, max_input_length=512, max_length=128)
test_dataset = Sum_Dataset(cleaned_ds['test'], tokenizer, max_input_length=512, max_length=128)

print(train_dataset[0])
print(validation_dataset[0])
print(test_dataset[0])
# Now you can use these datasets in your DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=4)
test_dataloader = DataLoader(test_dataset, batch_size=4)


{'input_ids': tensor([    3,    40,   106,  2029,     3,  4606,    40,   232,     3,  3272,
          651,   815,   449,  2213,     3,    26,  2738,    15,    40,     3,
         5672, 12591,    15, 11391,   592,    12,     3,     9,  2196,   770,
        13462,    38,     3,    88,  5050,    30,  1911,  1135,    68,     3,
           88, 10419,     7,     8,   540,    56,    59,  4061,     3,     9,
        10783,    30,   376,     3,    26,  2738,    15,    40,     3,  5672,
        12591,    15,    38,     3,  3272,   651,   815,   449,    16,     3,
         3272,   651,   815,   449,    11,     8,   455,    13,     8,     3,
         9553,    35,  2407,    12,     8, 19142,    13, 29517,  6710,   343,
            7,   300,     8,   296,     8,  1021,  7556,   845,     3,    88,
           65,   150,  1390,    12,  9030,    17,   449,   112,  1723,   550,
           30,  1006,  2948,  3281,    11, 17086,  2251,     3,    23,   103,
           59,   515,    12,    36,    80,    13, 

<ipython-input-8-99342543cb6a>:50: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "input_ids": torch.tensor(inputs["input_ids"]).squeeze(0),
<ipython-input-8-99342543cb6a>:51: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "attention_mask": torch.tensor(inputs["attention_mask"]).squeeze(0),
<ipython-input-8-99342543cb6a>:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "labels": torch.tensor(targets["input_ids"]).squeeze(0)  # Tokenized summaries as labels


In [ ]:
pip install rouge_score

In [ ]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.9 MB/s eta 0:00:00


In [ ]:

torch.cuda.empty_cache()



In [ ]:
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import PegasusTokenizer, PegasusForConditionalGeneration, AdamW, get_scheduler
import evaluate


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = T5Tokenizer.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained("t5-base")
model = model.to(device)


optimizer = AdamW(model.parameters(), lr=5e-5)
scaler = torch.cuda.amp.GradScaler()


num_epochs = 3
train_steps_per_epoch = len(train_dataloader)
total_training_steps = num_epochs * train_steps_per_epoch
lr_scheduler = get_scheduler(
    "linear", optimizer=optimizer, num_warmup_steps=500, num_training_steps=total_training_steps
)


rouge = evaluate.load("rouge")


best_loss = float("inf")
for epoch in range(num_epochs):
    model.train()
    progress_bar = tqdm(train_dataloader, desc=f"Training Epoch {epoch + 1}", total=len(train_dataloader))
    for step, batch in enumerate(progress_bar):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device).long()

        with torch.cuda.amp.autocast():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()
        lr_scheduler.step()

        progress_bar.set_postfix(loss=loss.item())

        if step % 500 == 0:
            print(f"Epoch {epoch + 1}, Step {step}, Loss: {loss.item()}")

    # Validation
    model.eval()
    eval_loss = 0.0
    rouge_score = rouge.compute()
    with torch.no_grad():
        for batch in validation_dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device).long()

            # Validation loss
            with torch.cuda.amp.autocast():
                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                eval_loss += outputs.loss.item() * input_ids.size(0)

            # ROUGE metric
            generated_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=128)
            decoded_preds = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
            decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
            rouge.add_batch(predictions=decoded_preds, references=decoded_labels)


    avg_eval_loss = eval_loss / len(validation_dataloader.dataset)
    print(f"Epoch {epoch + 1}, Validation Loss: {avg_eval_loss}")
    rouge_score = rouge.compute()
    print(f"Epoch {epoch + 1}, ROUGE Scores: {rouge_score}")


    if avg_eval_loss < best_loss:
        best_loss = avg_eval_loss
        model.save_pretrained("./best_model")
        tokenizer.save_pretrained("./best_model")

# Save the final model
model.save_pretrained("./final_model")
tokenizer.save_pretrained("./final_model")


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

# Load the model and tokenizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the saved model checkpoint
checkpoint_path = "/content/model_checkpoint.pth"
checkpoint = torch.load(checkpoint_path)

# Load the model and optimizer state dicts
model = T5ForConditionalGeneration.from_pretrained("t5-base")  #
model.load_state_dict(checkpoint['model_state_dict'])
model.to(device)

tokenizer = T5Tokenizer.from_pretrained("t5-base")  # or t5-large

# Set the model to evaluation mode
model.eval()

def summarize_text(text, max_input_length=512, max_output_length=150):
    # Add a summarization task prefix
    input_text = "summarize: " + text

    # Tokenize input text
    inputs = tokenizer(input_text, max_length=max_input_length, truncation=True, padding="max_length", return_tensors="pt")

    # Move tensors to the appropriate device
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    # Generate the summary
    with torch.no_grad():
        summary_ids = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_length=max_output_length,
            num_beams=4,  # Use beam search for better results
            length_penalty=1.5,  # Penalize longer outputs
            early_stopping=True
        )

    # Decode the summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# Test the model with a sample input
test_input = """
In the modern world, technology has had an incredibly large impact on various aspects of human life, reshaping society in ways that were once unimaginable.
From the early days of industrialization to the present-day digital revolution, technological advancements have transformed the way we work, communicate, and interact with the world.
One of the most significant technological innovations of the past century has been the advent of the internet, which has connected the entire world in ways that were once thought impossible.
The rise of the internet has revolutionized business, commerce, entertainment, education, and communication, making it easier for people to connect with others, access information, and conduct transactions across the globe.
"""

# Generate the summary
generated_summary = summarize_text(test_input)

print("Original Text: \n", test_input)
print("\nGenerated Summary: \n", generated_summary)


<ipython-input-12-ecaf455b4a4f>:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(checkpoint_path)
/usr/local/lib/python3.10/dist-packages/huggingface

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Original Text: 
 
In the modern world, technology has had an incredibly large impact on various aspects of human life, reshaping society in ways that were once unimaginable.
From the early days of industrialization to the present-day digital revolution, technological advancements have transformed the way we work, communicate, and interact with the world.
One of the most significant technological innovations of the past century has been the advent of the internet, which has connected the entire world in ways that were once thought impossible.
The rise of the internet has revolutionized business, commerce, entertainment, education, and communication, making it easier for people to connect with others, access information, and conduct transactions across the globe.


Generated Summary: 
 technology has had an incredibly large impact on various aspects of human life, reshaping society in ways that were once unimaginable. from the early days of industrialization to the present-day digital re

In [ ]:
!pip install rouge-score

In [ ]:

from rouge_score import rouge_scorer

# Function to calculate ROUGE score
def calculate_rouge_score(reference, summary):
    # Create a ROUGE scorer
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

    # Compute ROUGE scores
    scores = scorer.score(reference, summary)

    return scores

# Example reference summary (ground truth)
reference_summary = """
Technology has significantly impacted society, transforming the way people work, communicate, and interact with the world.
The internet and social media have revolutionized business, commerce, education, and communication.
However, challenges like privacy, security, job displacement, and the ethical implications of AI need careful consideration.
The future of work will be influenced by AI and automation, reshaping industries and job markets.
"""
test_input = """
In the modern world, technology has had an incredibly large impact on various aspects of human life, reshaping society in ways that were once unimaginable.
From the early days of industrialization to the present-day digital revolution, technological advancements have transformed the way we work, communicate, and interact with the world.
One of the most significant technological innovations of the past century has been the advent of the internet, which has connected the entire world in ways that were once thought impossible.
The rise of the internet has revolutionized business, commerce, entertainment, education, and communication, making it easier for people to connect with others, access information, and conduct transactions across the globe.
"""
# Generate summary
summary = summarize_text(test_input)

# Calculate ROUGE score
rouge_scores = calculate_rouge_score(reference_summary, summary)

# Print ROUGE scores
print(f"ROUGE-1: {rouge_scores['rouge1']}")
print(f"ROUGE-2: {rouge_scores['rouge2']}")
print(f"ROUGE-L: {rouge_scores['rougeL']}")
